First, we imports some useful packages

In [1]:
from paddle.trainer.PyDataProvider2 import *
from paddle.trainer_config_helpers import *
from py_paddle.trainer import *
import numpy

Secondly, we define networks.

In [2]:
@network(
    inputs={
        'pixel': dense_vector(784),
        'label': integer_value(10),
    },
    learning_rate=1e-4,
    learning_method=AdamOptimizer(),
    batch_size=1000)
def mnist_network(pixel, label):
    hidden1 = fc_layer(input=pixel, size=200)
    hidden2 = fc_layer(input=hidden1, size=200)
    inference = fc_layer(input=hidden2, size=10, act=SoftmaxActivation())
    cost = classification_cost(input=inference, label=label)
    return cost

what network do we have?

In [3]:
mnist = mnist_network()
print dir(mnist)

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__inputs__', '__module__', '__network_graph__', '__new__', '__optimize_graph__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'input_order', 'input_types', 'network_graph', 'optimize_graph']


In [4]:
print mnist.network_graph()

type: "nn"
layers {
  name: "pixel"
  type: "data"
  size: 784
  active_type: ""
}
layers {
  name: "label"
  type: "data"
  size: 10
  active_type: ""
}
layers {
  name: "__fc_layer_0__"
  type: "fc"
  size: 200
  active_type: "tanh"
  inputs {
    input_layer_name: "pixel"
    input_parameter_name: "___fc_layer_0__.w0"
  }
  bias_parameter_name: "___fc_layer_0__.wbias"
}
layers {
  name: "__fc_layer_1__"
  type: "fc"
  size: 200
  active_type: "tanh"
  inputs {
    input_layer_name: "__fc_layer_0__"
    input_parameter_name: "___fc_layer_1__.w0"
  }
  bias_parameter_name: "___fc_layer_1__.wbias"
}
layers {
  name: "__fc_layer_2__"
  type: "fc"
  size: 10
  active_type: "softmax"
  inputs {
    input_layer_name: "__fc_layer_1__"
    input_parameter_name: "___fc_layer_2__.w0"
  }
  bias_parameter_name: "___fc_layer_2__.wbias"
}
layers {
  name: "__cost_0__"
  type: "multi-class-cross-entropy"
  size: 1
  active_type: ""
  inputs {
    input_layer_name: "__fc_layer_2__"
  }
  inputs {
 

In [5]:
# Define a py data provider
@provider(
    input_types={'pixel': dense_vector(28 * 28),
                 'label': integer_value(10)},
    cache=CacheType.CACHE_PASS_IN_MEM)
def process(settings, filename):  # settings is not used currently.
    imgf = filename + "-images-idx3-ubyte"
    labelf = filename + "-labels-idx1-ubyte"
    f = open(imgf, "rb")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)

    # Define number of samples for train/test
    if "train" in filename:
        n = 60000
    else:
        n = 10000

    images = numpy.fromfile(
        f, 'ubyte', count=n * 28 * 28).reshape((n, 28 * 28)).astype('float32')
    images = images / 255.0 * 2.0 - 1.0
    labels = numpy.fromfile(l, 'ubyte', count=n).astype("int")

    for i in xrange(n):
        yield {"pixel": images[i, :], 'label': labels[i]}

    f.close()
    l.close()

In [6]:
print process

<class 'paddle.trainer.PyDataProvider2.DataProvider'>


In [7]:
runner = RunnerBuilder(network=mnist, device_count=2
                      ).with_std_local_trainer(method=process, 
                                               file_list=['./data/raw_data/train']
                                              ).with_std_tester(method=process,
                                                                file_list=['./data/raw_data/t10k']).build()
print runner

In [8]:
with runner:
    for _ in xrange(4):
        runner.run_one_pass()

Pass=0, Batch=0 Cost=2.576714, Eval: classification_error_evaluator=0.904 
Pass=0, Batch=1 Cost=2.453511, Eval: classification_error_evaluator=0.907 
Pass=0, Batch=2 Cost=2.386838, Eval: classification_error_evaluator=0.892 
Pass=0, Batch=3 Cost=2.372062, Eval: classification_error_evaluator=0.905 
Pass=0, Batch=4 Cost=2.291697, Eval: classification_error_evaluator=0.845 
Pass=0, Batch=5 Cost=2.251712, Eval: classification_error_evaluator=0.794 
Pass=0, Batch=6 Cost=2.204496, Eval: classification_error_evaluator=0.736 
Pass=0, Batch=7 Cost=2.186248, Eval: classification_error_evaluator=0.726 
Pass=0, Batch=8 Cost=2.169366, Eval: classification_error_evaluator=0.729 
Pass=0, Batch=9 Cost=2.142059, Eval: classification_error_evaluator=0.721 
Pass=0, Batch=10 Cost=2.094972, Eval: classification_error_evaluator=0.686 
Pass=0, Batch=11 Cost=2.086380, Eval: classification_error_evaluator=0.684 
Pass=0, Batch=12 Cost=2.065565, Eval: classification_error_evaluator=0.668 
Pass=0, Batch=13 Cost=

Done.